In [1]:
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import matplotlib.pyplot as plt
import numpy as np

In [45]:
#DATA AUGMENTATION 
def augment_images(image_path, filename):
		img = load_img(image_path)
		x = img_to_array(img)
		x = x.reshape((1, ) + x.shape)

		i = 0
		for batch in datagen.flow(x, batch_size = 16, shuffle=True, seed=1,
								save_to_dir ='C:/Users/Camille/Downloads/NEW PATH', #Camille
								# save_to_dir ='D:/Program Files/NEW CROPPED LABELED RESIZED PRES PICS', #Ash
								save_prefix =filename+'_'+str(i), save_format ='png'):
			i += 1
			if i > 5:
				break

datagen = ImageDataGenerator(					
		width_shift_range= 25,
		height_shift_range= 10,
		rotation_range = 5,
		shear_range = 1,
		zoom_range = 0.2)

# clean pics
input_path = 'C:/Users/Camille/Downloads/edited white' #Camille
# input_path = 'D:\Program Files\CROPPED LABELED RESIZED PRES PICS' #Ash
for filename in os.listdir(input_path):
	image_path = os.path.join(input_path, filename)
	augment_images(image_path, filename[:-4])

# dirty pics
input_path2 = 'C:/Users/Camille/Downloads/dirty pics' #Camille
for filename in os.listdir(input_path2):
	if filename.endswith('.png'):
		image_path = os.path.join(input_path2, filename)
		print(image_path)
		augment_images(image_path, filename[:-4])

In [46]:
# # Creating Train / Val / Test folders (One time use)
import os
import numpy as np
import shutil
import random
root_dir = 'C:/Users/Camille/Downloads/splitted data/' #Camille
# root_dir = 'D:/Program Files/SPLIT NEW CROPPED LABELED RESIZED PRES PICS/' #Ash

val_ratio = 0.15 #10
test_ratio = 0.15 #20 or #30!!

os.makedirs(root_dir +'train/')
os.makedirs(root_dir +'test/')
os.makedirs(root_dir +'val/')

# Creating partitions of the data after shuffeling
src = 'C:/Users/Camille/Downloads/NEW PATH' # Folder to copy images from --Camille
# src = 'D:/Program Files/NEW CROPPED LABELED RESIZED PRES PICS' #Ash

allFileNames = os.listdir(src)
random.seed(1)
np.random.shuffle(allFileNames)
train_FileNames, val_FileNames, test_FileNames = np.split(np.array(allFileNames),
                                                          [int(len(allFileNames)* (1 - (val_ratio + test_ratio))), 
                                                           int(len(allFileNames)* (1 - test_ratio))])


train_FileNames = [src+'/'+ name for name in train_FileNames.tolist()]
val_FileNames = [src+'/' + name for name in val_FileNames.tolist()]
test_FileNames = [src+'/' + name for name in test_FileNames.tolist()]

print('Total images: ', len(allFileNames))
print('Training: ', len(train_FileNames))
print('Validation: ', len(val_FileNames))
print('Testing: ', len(test_FileNames))

# Copy-pasting images
for name in train_FileNames:
    shutil.copy(name, root_dir +'train/' )
for name in test_FileNames:
    shutil.copy(name, root_dir +'test/' )
for name in val_FileNames:
    shutil.copy(name, root_dir +'val/' )

Total images:  2946
Training:  2062
Validation:  442
Testing:  442
